In [1]:
import numpy as np
import glob
import os
import pickle
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
#import stagnation.variables as va
from scipy.optimize import curve_fit
from scipy.spatial import cKDTree
from scipy.optimize import fsolve
import scipy
import time

In [2]:
L = 120 #box size
dx = 120/2**9 #minimum grid size
print(dx)
folderfig='/home/turbots/Documents/final_fig/R0/grad/'
eps = 7.8 #mean dissipation without the bubble
rho  = 1 #fluid density (taken as reference)
sig=dx/2

0.234375


In [3]:
def gaussian(x, sig): #fonction de Gauss utlisé pour la normalisation et le calcul des poids de chaque points
    return 1/(sig*np.sqrt(2*np.pi))*np.exp(-1/2*(x/sig)**2)

In [4]:
def expo_decr(x,a,b,cte):
    return np.exp(a*x+b)+cte

In [5]:
folds = glob.glob('Re*') # we pick up all the files that start with Re and put them in folds
len(folds)# lenght of folds

286

In [ ]:
folds0 = 'Re55t140-l9-16-2540-2-dump_63405'
filesdissint = glob.glob(folds0+ '/localstats_[!f]*')
filesdissext = glob.glob(folds0 + '/localstats_fluid*')
filesinterf = glob.glob(folds0 + '/inter*.dat')
        
    

import warnings

with warnings.catch_warnings():#to remove warnings associated to the fact that some files are empty.
    warnings.simplefilter("ignore") 
#     dataint = np.zeros([0, 7], dtype=float)
#     dataext = np.zeros([0, 6], dtype=float)
    interf = np.zeros([0, 10], dtype=float)
    bubble = np.zeros([0, 12], dtype=float)
    stat = np.zeros([0, 8], dtype=float)
    vit_in = np.zeros([0,17], dtype=float)
    vit_interf = np.zeros([0,17], dtype=float)
    vit_out = np.zeros([0,16], dtype=float)

    for file in filesinterf:
        datatemp = np.loadtxt(file, skiprows=1)
        if datatemp.shape !=(0,):
            interf = np.vstack((interf, datatemp))

try:
    datatemp = np.loadtxt(folds0 +'/bubbles.dat', skiprows=1)
except:
    datatemp = np.loadtxt(folds0 +'/bubbles.dat', skiprows=0)
if datatemp.shape !=(0,):
    bubble = np.vstack((bubble,datatemp))

    
datatemp=np.loadtxt(folds0 + '/vitessein_0.dat',skiprows=1)
if datatemp.shape !=(0,):
        vit_in = np.vstack((vit_in,datatemp))

datatemp=np.loadtxt(folds0 + '/vitesseinterf_0.dat',skiprows=1)
if datatemp.shape !=(0,):
        vit_interf = np.vstack((vit_interf,datatemp))

datatemp=np.loadtxt(folds0 + '/vitesseout_0.dat',skiprows=1)
if datatemp.shape !=(0,):
        vit_out = np.vstack((vit_out,datatemp))

    
    

In [ ]:
R0= float(folds0.split('-')[2])

In [6]:
params = np.zeros(9)

def surf(x, y, z):#equation of the surface
    global params
#     a2, a1, b2, b1, c2, c1, ab, ac, bc, d = params
    a1,b1 ,c1, ab , bc, ac , a2 , b2 , c2 = params
    return a2*x**2 + a1*x + b2*y**2 + b1*y + c2*z**2 + c1*z + ab*x*y + ac*x*z + bc*y*z

def gradient(x, y, z):#gradient of the surface 
    global params
#     a2, a1, b2, b1, c2, c1, ab, ac, bc, d = params
    a1, b1 ,c1, ab , bc, ac , a2 , b2 , c2 = params
    gx = 2*a2*x + a1 + ab*y+ ac*z
    gy = 2*b2*y + b1 + ab*x + bc*z
    gz = 2*c2*z + c1 + ac*x + bc*y
    return np.asarray([gx, gy, gz])

def system(XYZLamb, M):
    '''
    to solve:
    x - xM - lambda gx = 0
    y - yM - lambda gy = 0
    z - zM - lambda gz = 0
    surf(x, y, z) = 0
    
    ie: OM and gradient are colinear and O is on the surface.
    '''
    g = gradient(*XYZLamb[:3])
    p1 = M - XYZLamb[:3] - XYZLamb[3]*g
    p1 = np.append(p1, surf(*XYZLamb[:3]) )
    return p1

In [7]:
mask_vit_in=(vit_in[:,0]<600)
maskinterf=(interf[:,0]<600)

print(30*dx)
for bulle in bubble:
    if bulle[3]<30*dx: #la bulle est plus petite que 30 cellules ie pas une "bulle"
        j=bulle[2]
        mask_vit_in=np.logical_and(mask_vit_in,vit_in[:,0][:]!=j)
        maskinterf=np.logical_and(maskinterf,interf[:,0][:]!=j)

NameError: name 'vit_in' is not defined

In [ ]:
pc = np.median(vit_in[:, 1:4][mask_vit_in], axis=0)# approximate position center
print(pc)
interf[:, 1:4] = np.mod(interf[:, 1:4] - pc + L/2, L) - L/2 #shift interface position
vit_in[:, 1:4] = np.mod(vit_in[:, 1:4] - pc + L/2, L) - L/2 #shift data position
vit_interf[:, 1:4] = np.mod(vit_interf[:, 1:4] - pc + L/2, L) - L/2 #shift data position
vit_out[:, 0:3] = np.mod(vit_out[:, 0:3] - pc + L/2, L) - L/2 #shift data position
pc = np.mean(vit_in[:, 1:4][mask_vit_in], axis=0)# approximate position center
print(pc)

In [ ]:
# on masque les points extérieures de la bulle qui sont trop éloigné du centre de la bulle pour réduire le temps 
# de calcul et car on ne s'intéresse qu'a ce qu'il se passe proche de la bulle

mask_vit_out=((vit_out[:,0]-pc[0])**2+(vit_out[:,1]-pc[1])**2+(vit_out[:,2]-pc[2])**2<((3.5*R0)**2)) 

In [ ]:
z = np.mean(vit_in[:, 3][mask_vit_in])
dz=0.5
y=np.mean(vit_in[:, 2][mask_vit_in])
dy=0.5

indicez4= np.logical_and(vit_in[:, 3]>z-dz, vit_in[:, 3]<z+dz )
indicey4= np.logical_and(vit_in[:, 2]>y-dy, vit_in[:, 2]<y+dy )
mask_vit_in_1d=np.logical_and(indicey4,indicez4)
mask_vit_in_2d=np.logical_and(indicez4,mask_vit_in) #masque 2d pour le champ de données intérieures
# mask_vit_in_1d=np.logical_and(mask_vit_in_1d,mask_vit_in) #masque 1d pour le champ de données intérieures

indicez5= np.logical_and(vit_out[:, 2]>z-dz, vit_out[:, 2]<z+dz )
indicey5= np.logical_and(vit_out[:, 1]>y-dy, vit_out[:, 1]<y+dy )
mask_vit_out_2d=np.logical_and(mask_vit_out,indicez5)#masque 2d pour le champ de données extérieures
mask_vit_out_1d=np.logical_and(indicey5,indicez5)#masque 1d pour le champ de données extérieures
mask_vit_out_1d=np.logical_and(mask_vit_out,mask_vit_out_1d)

In [ ]:
fig = plt.figure(dpi=150)
ax = plt.axes(projection='3d')
ax.scatter(interf[:,1][maskinterf],interf[:,2][maskinterf],interf[:,3][maskinterf],color='k',s=1)
# plt.title('interface of the bubble')
# plt.savefig(folderfig + 'interface_solo_bubble.pdf',bbox_inches='tight')

In [ ]:
voronoi_kdtree = cKDTree(interf[:, 1:4]) #fonction qui trace un diagramme de Voronoi. 
#Ce diagrame permet de crée des régions où sont regorupés tout les points les points proches 
#d'un point de l'interface. Ça permet de calculer la distance des points à l'interface

In [ ]:
test_point_dist_vit_out, regions_ext = voronoi_kdtree.query(vit_out[:, 0:3][mask_vit_out])
test_point_dist_vit_in, regions_int = voronoi_kdtree.query(vit_in[mask_vit_in][:, 1:4])

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')
#np.mod(X - pc + L/2, L) - L/2
p=ax.scatter(vit_out[:, 0][mask_vit_out], vit_out[:, 1][mask_vit_out], vit_out[:, 2][mask_vit_out],c=regions_ext, s=1)
plt.title('Voronoi spaces outside the bubble')
# ax.scatter(interf[:,1][maskinterf],interf[:,2][maskinterf],interf[:,3][maskinterf],color='r')
ax.set_xlabel('X ')
ax.set_ylabel('Y ')
ax.set_zlabel('Z ')

In [18]:
def interpolate_surf(interface_points,voronoi_kdtree):
#     calcule des coefficients d'interpolation de la surf pour tous les points de l'interface
    surface = {}
    for i , element in enumerate(interface_points):
        (distances,voisins) = voronoi_kdtree.query(element,k=20)
        vois = interface_points[voisins,:]
        A = np.c_[vois[:,:], vois[:,0]*vois[:,1],vois[:,2]*vois[:,1],vois[:,0]*vois[:,2], vois[:,:]**2]
        params,_,_,_ = scipy.linalg.lstsq(A, np.ones(vois.shape))
        params = params[:,0]
        surface[i] = params
    return surface

In [14]:
def compute_dist_norm(points, interface_points, regions, surface):
# calcule de la distance à l'interface pour tous les points mis en arguments dans 'points'
    distance =  np.zeros(len(points))
    normale = np.zeros([len(points),3])
    for i , element in enumerate(points):
        indice_interface = regions[i]
        interf = interface_points[indice_interface]
        params = surface[indice_interface]
        out = fsolve(system, np.append(interf+1,1), args=(element), full_output=True)
        sol = out[0]
        normal = sol[:3]/np.linalg.norm(sol[:3])
        n_voronoi = element - interf
        distance[i] = np.sum(normal*n_voronoi)
        normale[i] = normal
    return distance, normale

In [15]:
def calculate_orthogonal_vectors(v): #calculer les deux vecteurs orthonormaux à un vecteur v
    
    
    # Générer un vecteur aléatoire de même dimension que v
    random_vector = np.random.randn(len(v))
    
    # Calculer le produit vectoriel entre v et le vecteur aléatoire
    cross_product = np.cross(v, random_vector)
    
    # Calculer le produit vectoriel entre v et le produit vectoriel précédent
    orthogonal_vector = np.cross(v, cross_product)
    
    # Normaliser les vecteurs
    cross_product_normalized = cross_product / np.linalg.norm(cross_product)
    orthogonal_vector_normalized = orthogonal_vector / np.linalg.norm(orthogonal_vector)
    
    return cross_product_normalized, orthogonal_vector_normalized

In [ ]:
tps1 = time.time()
surface = interpolate_surf(interf[:,1:4][maskinterf],voronoi_kdtree)
distance_ext, normale_ext = compute_dist_norm(vit_out[mask_vit_out][:,:3],interf[:,1:4][maskinterf],regions_ext,surface)
distance_int, normale_int = compute_dist_norm(vit_in[mask_vit_in][:,1:4],interf[:,1:4][maskinterf],regions_int,surface)

tps2 = time.time()
print(tps2-tps1)

In [ ]:
t1 = time.time()

tangente1_ext, tangente2_ext = np.zeros(normale_ext.shape), np.zeros(normale_ext.shape)

for (i,element) in enumerate(normale_ext):
    tangente1_ext[i], tangente2_ext[i] = calculate_orthogonal_vectors(element)
    
tangente1_int, tangente2_int = np.zeros(normale_int.shape), np.zeros(normale_int.shape)

for (i,element) in enumerate(normale_int):
    tangente1_int[i], tangente2_int[i] = calculate_orthogonal_vectors(element)

t2 = time.time()
print(t2-t1)

In [ ]:
def proj_vit(vitesse,normale):
    normv = np.abs(np.sum(vitesse[:]*normale[:],axis=1))
    
    v_n = np.zeros([len(normv),3])
    v_n[:,0] = normv*normale[:,0]
    v_n[:,1] = normv*normale[:,1]
    v_n[:,2] = normv*normale[:,2]
    
    tangv = np.linalg.norm(vitesse[:] - v_n[:],axis=1)
    
    return normv, tangv

In [ ]:
t1 = time.time()
normv_out,tangv_out = proj_vit(vit_out[:,3:6][mask_vit_out],normale_ext)
normv_in,tangv_in = proj_vit(vit_in[:,4:7][mask_vit_in],normale_int)
t2 = time.time()
print(t2-t1)

In [25]:
def proj_grad_out(vitesse,normale,tangente1,tangente2):
    
    gradv_n=np.zeros([len(vitesse),3])
    gradv_n[:,0]=np.sum(vitesse[:,6:9]*normale[:,0:],axis=1)
    gradv_n[:,1]=np.sum(vitesse[:,9:12]*normale[:,0:],axis=1)
    gradv_n[:,2]=np.sum(vitesse[:,12:15]*normale[:,0:],axis=1)
    normgdn=np.sum(gradv_n[:,0:3]**2,axis=1)


    gradv_t1=np.zeros([len(vitesse),3])
    gradv_t1[:,0]=np.sum(vitesse[:,6:9]*tangente1[:,0:],axis=1)
    gradv_t1[:,1]=np.sum(vitesse[:,9:12]*tangente1[:,0:],axis=1)
    gradv_t1[:,2]=np.sum(vitesse[:,12:15]*tangente1[:,0:],axis=1)
    normgdt1=np.sum(gradv_t1[:,0:3]**2,axis=1)


    gradv_t2=np.zeros([len(vitesse),3])
    gradv_t2[:,0]=np.sum(vitesse[:,6:9]*tangente2[:,0:],axis=1)
    gradv_t2[:,1]=np.sum(vitesse[:,9:12]*tangente2[:,0:],axis=1)
    gradv_t2[:,2]=np.sum(vitesse[:,12:15]*tangente2[:,0:],axis=1)
    normgdt2=np.sum(gradv_t2[:,0:3]**2,axis=1)

    
    return normgdn , normgdt1 , normgdt2

In [26]:
def proj_grad_in(vitesse,normale,tangente1,tangente2):
    
    gradv_n=np.zeros([len(vitesse),3])
    gradv_n[:,0]=np.sum(vitesse[:,7:10]*normale[:,0:],axis=1)
    gradv_n[:,1]=np.sum(vitesse[:,10:13]*normale[:,0:],axis=1)
    gradv_n[:,2]=np.sum(vitesse[:,13:16]*normale[:,0:],axis=1)
    normgdn=np.sum(gradv_n[:,0:3]**2,axis=1)


    gradv_t1=np.zeros([len(vitesse),3])
    gradv_t1[:,0]=np.sum(vitesse[:,7:10]*tangente1[:,0:],axis=1)
    gradv_t1[:,1]=np.sum(vitesse[:,10:13]*tangente1[:,0:],axis=1)
    gradv_t1[:,2]=np.sum(vitesse[:,13:16]*tangente1[:,0:],axis=1)
    normgdt1=np.sum(gradv_t1[:,0:3]**2,axis=1)


    gradv_t2=np.zeros([len(vitesse),3])
    gradv_t2[:,0]=np.sum(vitesse[:,7:10]*tangente2[:,0:],axis=1)
    gradv_t2[:,1]=np.sum(vitesse[:,10:13]*tangente2[:,0:],axis=1)
    gradv_t2[:,2]=np.sum(vitesse[:,13:16]*tangente2[:,0:],axis=1)
    normgdt2=np.sum(gradv_t2[:,0:3]**2,axis=1)

    
    return normgdn , normgdt1 , normgdt2

In [ ]:
t1 = time.time()

gdn_out, gdt1_out, gdt2_out = proj_grad_out(vit_out[mask_vit_out],normale_ext,tangente1_ext,tangente2_ext)
gdn_in, gdt1_in , gdt2_in = proj_grad_in(vit_in[mask_vit_in],normale_int,tangente1_int,tangente2_int)

t2 = time.time()
print(t2-t1)

In [ ]:
print( gdn_out)

In [ ]:
dist_int = np.linspace(0,R0,100)
dist_ext = np.linspace(0,2.5*R0,350)

In [ ]:
#test de la normalisation avec une fonction exponentielle


weights = gaussian(distance_ext -dist_ext[:, np.newaxis], sig)

expo=np.exp(distance_ext)

mean = np.sum(expo*weights*vit_out[:, -1][mask_vit_out], axis=1)/np.sum(vit_out[:, -1][mask_vit_out]*weights, axis=1)


In [ ]:
plt.figure(dpi=150)
plt.plot(dist_ext,mean)
plt.xlabel('$d$')
plt.ylabel('$exp(d)$')
plt.plot(dist_ext,np.exp(dist_ext),color='r',linestyle='--')
plt.title('test normalisation avec une fonction exponentielle')
# plt.savefig(folderfig+'test_normalisation.pdf')
print(mean[0])

In [ ]:
weights = gaussian(distance_ext -dist_ext[:, np.newaxis], sig)
vn_out = np.sum(normv_out*weights*vit_out[:, -1][mask_vit_out], axis=1)/np.sum(vit_out[:, -1][mask_vit_out]*weights,axis=1)
vt_out = np.sum(tangv_out*weights*vit_out[mask_vit_out][:, -1], axis=1)/np.sum(vit_out[mask_vit_out][:, -1]*weights,axis=1)

weights = gaussian(distance_int -dist_int[:, np.newaxis], sig)
vn_in = np.sum(normv_in*weights*vit_in[:, -1][mask_vit_in], axis=1)/np.sum(vit_in[:, -1][mask_vit_in]*weights,axis=1)
vt_in = np.sum(tangv_in*weights*vit_in[:, -1][mask_vit_in], axis=1)/np.sum(vit_in[:, -1][mask_vit_in]*weights,axis=1)


In [ ]:
plt.figure(dpi = 150)
plt.plot(dist_int*(-1)/R0, vn_in, color='g')
plt.plot(dist_int*(-1) /R0, vt_in/2, color='b')

plt.plot(dist_ext/R0, vn_out, color='g')
plt.plot(dist_ext/R0 , vt_out/2, color='b')



In [ ]:
weights = gaussian(distance_ext -dist_ext[:, np.newaxis], sig)
gdvn_out = np.sum(gdn_out*weights*vit_out[:, -1][mask_vit_out], axis=1)/np.sum(vit_out[:, -1][mask_vit_out]*weights,axis=1)
gdvt1_out = np.sum(gdt1_out*weights*vit_out[mask_vit_out][:, -1], axis=1)/np.sum(vit_out[mask_vit_out][:, -1]*weights,axis=1)
gdvt2_out = np.sum(gdt2_out*weights*vit_out[mask_vit_out][:, -1], axis=1)/np.sum(vit_out[mask_vit_out][:, -1]*weights,axis=1)

weights = gaussian(distance_int -dist_int[:, np.newaxis], sig)
gdvn_in = np.sum(gdn_in*weights*vit_in[:, -1][mask_vit_in], axis=1)/np.sum(vit_in[:, -1][mask_vit_in]*weights,axis=1)
gdvt1_in = np.sum(gdt1_in*weights*vit_in[:, -1][mask_vit_in], axis=1)/np.sum(vit_in[:, -1][mask_vit_in]*weights,axis=1)
gdvt2_in = np.sum(gdt2_in*weights*vit_in[:, -1][mask_vit_in], axis=1)/np.sum(vit_in[:, -1][mask_vit_in]*weights,axis=1)


In [ ]:
plt.figure(dpi = 150)
plt.plot(dist_int*(-1)/R0, gdvn_in**2, color='g')
plt.plot(dist_int*(-1) /R0, (gdvt1_in**2+gdvt2_in**2)/2, color='b')

plt.plot(dist_ext/R0, gdvn_out**2, color='g')
plt.plot(dist_ext/R0 , (gdvt1_out**2+gdvt2_out**2)/2, color='b')

In [8]:
def loadfiles(filename):
    '''
    Load vitessein, vitesseout, vitesseinterf and interface files knowing the folder direction. 
    Return velocity fields arrays
    '''
    import warnings

    with warnings.catch_warnings():#to remove warnings associated to the fact that some files are empty.
        warnings.simplefilter("ignore")    

        filesinterf = glob.glob(filename + '/inter*.dat')
        
        interf = np.zeros([0, 10], dtype=float)
        bubble = np.zeros([0,12], dtype=float)
        vit_in = np.zeros([0,15], dtype=float)
        vit_interf = np.zeros([0,15], dtype=float)
        vit_out = np.zeros([0,15], dtype=float)
        
        for file in filesinterf:
            datatemp = np.loadtxt(file, skiprows=1)
            if datatemp.shape !=(0,):
                interf = np.vstack((interf, datatemp))
        
        datatemp=np.loadtxt(filename + '/bubbles.dat',skiprows=1)
        if datatemp.shape !=(0,):
                bubble = np.vstack((bubble,datatemp))

        vit_in=np.loadtxt(filename + '/vitessein_0.dat',skiprows=1)

        vit_interf=np.loadtxt(filename + '/vitesseinterf_0.dat',skiprows=1)

        vit_out=np.loadtxt(filename + '/vitesseout_0.dat',skiprows=1)


        return interf, bubble, vit_in, vit_interf, vit_out

In [ ]:

vitesseext = {}
vitesseint = {}
# vitesseinterf = {}
occ_ro_dump = {}

for fold in folds:
    R0 = float(fold.split('-')[2])
    dist_int = np.linspace(0,R0,100)
    dist_ext = np.linspace(0,2*R0,250)
    interf, bubble, vit_in, vit_interf, vit_out = loadfiles(fold);
    #test whether the bubble broke or not. If so there are several bubbles and so 
    # bubble number can be larger than 0. 
    if int(fold.split('-')[2])==8:
        if int(fold.split('-')[3])!=800:
            continue
    print(fold)
    if 1 in vit_in[:, 0]:
        print('break')
        print(np.max(data[:, 0]))
        continue
    
    

    #recenter

#     point intérieurs
    maskinterf=(interf[:,0]>-1)
    mask_vit_in=(vit_in[:,0]<600)
    mask_vit_interf=(vit_interf[:,0]<600)

#     print(bubble)
    for bulle in bubble:
        if bulle[3]<30*dx: #la bulle est plus petite que 30 cellules ie pas une "bulle"
            j=bulle[2]
            maskinterf=np.logical_and(maskinterf,interf[:,0][:]!=j)
            mask_vit_in=np.logical_and(mask_vit_in,vit_in[:,0][:]!=j)
            mask_vit_interf=np.logical_and(mask_vit_interf,vit_interf[:,0][:]!=j)
    # on recentre les points
    pc = np.median(vit_in[mask_vit_in][:,1:4])
    interf[:, 1:4] = np.mod(interf[:, 1:4] - pc + L/2, L) - L/2 #shift interface position
    vit_in[:, 1:4] = np.mod(vit_in[:, 1:4] - pc + L/2, L) - L/2 #shift data position
    vit_interf[:, 1:4] = np.mod(vit_interf[:, 1:4] - pc + L/2, L) - L/2 #shift data position
    vit_out[:, 0:3] = np.mod(vit_out[:, 0:3] - pc + L/2, L) - L/2 #shift data position
    #compute distance
    
    #Voronoi diagram
    voronoi_kdtree = cKDTree(interf[maskinterf][:,1:4])
    surface = interpolate_surf(interf[maskinterf][:,1:4],voronoi_kdtree)
    #Region of the test points
    test_point_dist, regions_int = voronoi_kdtree.query(vit_in[:,1:4][mask_vit_in])
#     mask_vit_in = np.logical_and(mask_vit_in,positif)
    distance_int, normale_int = compute_dist_norm( vit_in[:,1:4][mask_vit_in], interf[maskinterf][:,1:4], regions_int,surface)
    
    
    tangente1_int, tangente2_int = np.zeros([len(normale_int),3]), np.zeros([len(normale_int),3])
    for (i,n) in enumerate(normale_int):
            tangente1_int[i], tangente2_int[i] = calculate_orthogonal_vectors(n)
    

    gdn_in, gdt1_in, gdt2_in = np.zeros(len(normale_int)), np.zeros(len(normale_int)), np.zeros(len(normale_int))
    gdn_in, gdt1_in , gdt2_in = proj_grad_in(vit_in[mask_vit_in],normale_int,tangente1_int,tangente2_int)


    weights = gaussian(distance_int -dist_int[:, np.newaxis], sig)
    gdvn_in = np.sum(gdn_in*weights*vit_in[:, -1][mask_vit_in], axis=1)/np.sum(vit_in[:, -1][mask_vit_in]*weights,axis=1)
    gdvt1_in = np.sum(gdt1_in*weights*vit_in[:, -1][mask_vit_in], axis=1)/np.sum(vit_in[:, -1][mask_vit_in]*weights,axis=1)
    gdvt2_in = np.sum(gdt2_in*weights*vit_in[:, -1][mask_vit_in], axis=1)/np.sum(vit_in[:, -1][mask_vit_in]*weights,axis=1)

    
    r0 = float(fold.split('-')[2])#bubble size

    if r0 not in vitesseint:
        vitesseint[r0] = np.zeros([4, len(dist_int)])
        occ_ro_dump[r0] = 0

    ind = dist_int<np.max(test_point_dist) 
#     ind = np.logical_and(ind, gdvn_in>0) 
    
    vitesseint[r0][0, :][ind] += gdvn_in[ind]
    vitesseint[r0][1, :][ind] += gdvt1_in[ind]
    vitesseint[r0][2, :][ind] += gdvt2_in[ind]
    vitesseint[r0][-1,:][ind] += 1
    occ_ro_dump[r0]+=1
    
    with open( fold +  '/vitint.pkl', 'wb') as f:
        pickle.dump(vitesseint, f)
    
    
    
    #     points extérieurs
    gamma = float(fold.split('-')[2])#bubble size
    pc=np.mean(vit_in[:,1:4], axis=0)
    mask_vit_out=((vit_out[:,0]-pc[0])**2+(vit_out[:,1]-pc[1])**2+(vit_out[:,2]-pc[2])**2<(25*gamma**2)) #5*R0
    #compute distance
    distpoints_vit, regions_ext = voronoi_kdtree.query(vit_out[:,0:3][mask_vit_out])
    distance_ext, normale_ext = compute_dist_norm(vit_out[:,:3][mask_vit_out],interf[maskinterf][:,1:4], regions_ext,surface)

    tangente1_ext, tangente2_ext = np.zeros([len(normale_ext),3]), np.zeros([len(normale_ext),3])
    for (i,n) in enumerate(normale_ext):
            tangente1_ext[i], tangente2_ext[i] = calculate_orthogonal_vectors(n)
            
    gdn_out, gdt1_out , gdt2_out = np.zeros(len(normale_ext)),np.zeros(len(normale_ext)), np.zeros(len(normale_ext))
    gdn_out, gdt1_out, gdt2_out = proj_grad_out(vit_out[mask_vit_out],normale_ext,tangente1_ext,tangente2_ext)

    
    weights = gaussian(distance_ext -dist_ext[:, np.newaxis], sig)
    gdvn_out = np.sum(gdn_out*weights*vit_out[:, -1][mask_vit_out], axis=1)/np.sum(vit_out[:, -1][mask_vit_out]*weights,axis=1)
    gdvt1_out = np.sum(gdt1_out*weights*vit_out[mask_vit_out][:, -1], axis=1)/np.sum(vit_out[mask_vit_out][:, -1]*weights,axis=1)
    gdvt2_out = np.sum(gdt2_out*weights*vit_out[mask_vit_out][:, -1], axis=1)/np.sum(vit_out[mask_vit_out][:, -1]*weights,axis=1)


    
    if gamma not in vitesseext:
        vitesseext[gamma] = np.zeros([4, len(dist_ext)])


    ind = dist_ext<np.max(distpoints_vit) 
    vitesseext[r0][0, :][ind] += gdvn_out[ind]
    vitesseext[r0][1, :][ind] += gdvt1_out[ind]
    vitesseext[r0][2, :][ind] += gdvt2_out[ind]
    vitesseext[r0][-1,:][ind] += 1
    
    with open( fold +  '/vitext.pkl', 'wb') as f:
        pickle.dump(vitesseext, f)
    
for tab in [vitesseext,vitesseint]:
    for key in tab:
        tab[key][0:-1, :] /= np.where(tab[key][-1, :]>0, tab[key][-1, :], 1)

Re55t140-l9-16-2540-2-dump_63405


In [ ]:
print(gdn_in, gdt1_in, gdt2_in )

In [ ]:
print(normv_out, tangv_out)

In [ ]:
print(gdn_out, gdt1_out, gdt2_out )